# TP2 - Modeling using docplex

## 1. The `docplex` python package

`DOcplex` is a python package developped by IBM &mdash; It provides easy-to-use API for IBM solvers Cplex and Cpoptimizer.

DOcplex documentation for mathematical programming can be found here: http://ibmdecisionoptimization.github.io/docplex-doc/#mathematical-programming-modeling-for-python-using-docplex-mp-docplex-mp

## 2. Solving TSP using `docplex`

### 2.1. TSP model using `docplex`

**Exercice:** Using `docplex`, create a model for the travelling salesman problem using the MTZ or Flow formulation and compare them.

In [8]:
from docplex.mp.model import Model
import tsp.data as data

distances = data.grid42
#N = len(distances)
N = 20 #promotional version

tsp = Model("TSP")
tsp.log_output = True

#Variables
x = [tsp.binary_var_list(N, name=f"x{i}") for i in range(N)]
y = [[x[i][j] for i in range(N)] for j in range(N)]
u = tsp.integer_var_list(N, [1,*[2]*(N-1)], N)

#Constraints
tsp.add_constraint(u[0] == 1)
for i in range(N):
    tsp.add_constraint(x[i][i] == 0)
    tsp.add_constraint(sum(x[i][:i] + x[i][i+1:])==1)
    tsp.add_constraint(sum(y[i][:i] + y[i][i+1:])==1)

for i in range(1, N):
    for j in range(1, N):
        if i != j:
            tsp.add_constraint((u[i]-u[j]+1) <= ((N-1)*(1-x[i][j]))) 

#Objective
tsp.minimize(sum([tsp.dot(x[i], distances[i]) for i in range(N)]))


solution = tsp.solve()
print("z* =", solution.objective_value)

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 21 rows and 21 columns.
Reduced MIP has 382 rows, 399 columns, and 1786 nonzeros.
Reduced MIP has 380 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.17 ticks)
Probing time = 0.02 sec. (1.60 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 382 rows, 399 columns, and 1786 nonzeros.
Reduced MIP has 380 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.49 ticks)
Probing time = 0.00 sec. (1.62 ticks)
Clique table members: 211.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.64 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*   

 367205 57811        cutoff            518.0000      501.0000  2866536    3.28%
 378668 55524      509.5000    14      518.0000      502.0000  2945742    3.09%
 390177 52660        cutoff            518.0000      502.6667  3040066    2.96%
 401957 48916        cutoff            518.0000      503.5833  3133266    2.78%
 414083 45224      506.8333    20      518.0000      504.4706  3224036    2.61%
 426310 40671      507.6667    11      518.0000      505.4444  3314405    2.42%
 438717 35245      508.2639    25      518.0000      506.5000  3414282    2.22%
 451284 28370    infeasible            518.0000      508.0000  3520183    1.93%
 463987 19305        cutoff            518.0000      510.0000  3639191    1.54%
 477237 10736        cutoff            518.0000      512.0000  3729714    1.16%
Elapsed time = 88.13 sec. (41695.96 ticks, tree = 6.48 MB, solutions = 9)

GUB cover cuts applied:  70
Clique cuts applied:  9
Cover cuts applied:  77
Implied bound cuts applied:  5
Mixed integer roun

The largest set of distances contains 42 nodes, and should be easily solved by `docplex`.

### 2.2. Generating random TSP instances

**Question:** What method could you implement to generate a realistic set of distances for $n$ customers?

**Exercice:** Implement the method proposed above and test it.

In [11]:
import numpy as np
from scipy.spatial import distance_matrix


def generate_distances(n: int):
    x = (np.random.rand(n,2) * 100).astype(int)
    return distance_matrix(x,x,1)


from docplex.mp.model import Model

#distances = generate_distances(50)
distances = generate_distances(25) #promotional version
print(distances)

N = len(distances)
tsp = Model("TSP")
tsp.log_output = True

#Variables
x = [tsp.binary_var_list(N, name = f"X_{i}") for i in range(N)]
y = [[x[i][j] for i in range(N)] for j in range(N)]
u = tsp.integer_var_list(N, [1,*[2]*(N-1)], N)

#Constraints
tsp.add_constraint(u[0] == 1)
for i in range(N):
    tsp.add_constraint(x[i][i] == 0)
    tsp.add_constraint(tsp.sum(x[i][:i] + x[i][i+1:]) == 1)
    tsp.add_constraint(tsp.sum(y[i][:i] + y[i][i+1:]) == 1)

for i in range(1, N):
    for j in range(1, N):
        if i != j:
            tsp.add_constraint((u[i]-u[j]+1) <= ((N-1)*(1-x[i][j])))

#Objective
tsp.minimize(sum([tsp.dot(x[i], distances[i]) for i in range(N)]))


solution = tsp.solve()
print("z* =", solution.objective_value)

[[  0.  80.  91.  45.  46.  26.  26. 105.  97.  63.  32.  17.  56.  55.
   41.  91.  44.  73.  74.  97. 134.  65.  34.  52.  44.]
 [ 80.   0. 131.  81.  86.  90.  66. 109. 137.  23.  74.  63.  96.  95.
   75.  13.  42.  93.  10.  17.  66.  35.  46.  92.  36.]
 [ 91. 131.   0.  50.  59.  73.  65.  42.  30. 108.  59.  82.  35.  36.
   56. 118.  89.  38. 121. 116.  71.  96.  85.  39. 129.]
 [ 45.  81.  50.   0.  41.  55.  27.  60.  56.  58.  39.  32.  15.  14.
    6.  68.  39.  28.  71.  66.  89.  46.  35.  11.  79.]
 [ 46.  86.  59.  41.   0.  20.  20. 101.  51.  63.  14.  37.  36.  31.
   37.  87.  44.  69.  76.  93. 130.  61.  40.  36.  84.]
 [ 26.  90.  73.  55.  20.   0.  28. 115.  71.  73.  16.  27.  50.  45.
   51. 101.  54.  83.  84. 107. 144.  75.  44.  50.  64.]
 [ 26.  66.  65.  27.  20.  28.   0.  87.  71.  45.  12.  17.  30.  29.
   23.  73.  26.  55.  56.  79. 116.  47.  20.  26.  64.]
 [105. 109.  42.  60. 101. 115.  87.   0.  72.  86.  99.  88.  65.  70.
   64.  96.  67.  

*     0+    0                          514.0000      508.8686             1.00%
      0     0        cutoff            514.0000      514.0000      533    0.00%
Elapsed time = 0.45 sec. (119.42 ticks, tree = 0.01 MB, solutions = 6)

Clique cuts applied:  8
Implied bound cuts applied:  10
Mixed integer rounding cuts applied:  7
Zero-half cuts applied:  7
Multi commodity flow cuts applied:  22
Lift and project cuts applied:  1
Gomory fractional cuts applied:  7

Root node processing (before b&c):
  Real time             =    0.45 sec. (119.47 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.45 sec. (119.47 ticks)
z* = 514.0


## 3. Solving Warehouse Allocation using Benders decomposition with `docplex`

### 3.1. The warehouse problem

A company needs to supply a set of $n$ clients and needs to open new warehouses (from a
set of $m$ possible warehouses).
Opening a warehouse $j$ costs $f_j$ and supplying a client $i$ from a warehouse $j$ costs $c_{ij}$ per supply unit.
Which warehouses should be opened in order to satisfy all clients while minimizing the total cost?

### 3.2. Solving the warehouse problem with a single ILP

- $y_{j} = 1$ if and only if warehouse $j$ is opened.
- $x_{ij}$ is the fraction supplied from warehouse $j$ to customer $i$.

$
\begin{align}
  \text{min.} \quad & \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij} + \sum_{j=1}^{m} f_{j} y_{j} & \\
  \text{s.t.} \quad & \sum_{j=1}^{m} x_{ij} = 1, & \forall i \in\{1,\ldots,n\}\\
                    & x_{ij} \leq y_{j}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                    & y_{j} \in \left\{0,~1\right\}, & \forall j \in \left\{1,~\ldots,~m\right\}\\
                    & x_{ij} \geq 0, & \forall i \in \left\{1,~\ldots,~n\right\}, \forall j \in \left\{1,~\ldots,~m\right\}
\end{align}
$


**Exercice:** Implement the ILP model for the warehouse allocation problem and test it on the given instance.

In [12]:
from docplex.mp.model import Model

# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

wa = Model("Warehouse Allocation")
wa.log_output = True

#Variables
x = [wa.integer_var_list(N, 0, 1, f"X_{i}") for i in range(N)]
y = wa.binary_var_list(M, name="Y")

#Constraints
for i in range(N):
    wa.add_constraint(wa.sum(x[i]) == 1)
    for j in range(N):
        wa.add_constraint(x[i][j] <= y[j])

#Objective
wa.minimize(wa.sum([wa.dot(x[i], c[i]) for i in range(N)]) +wa.dot(y,f))

solution = wa.solve()
print("z* =", solution.objective_value)

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 12.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*

### 3.3. Benders' decomposition for the Warehouse Allocation problem

We are going to use Benders' decomposition to solve the Warehouse Allocation problem. 

#### Dual subproblem

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

#### Master problem

$
\begin{align*}
  \text{min.} \quad & \sum_{j=1}^{m} f_j y_j + z & \\
  \text{s.t.} \quad & z \geq \sum_{i=1}^{n}v_i^p - \sum_{i=1}^{n} \sum_{j=1}^{m} u_{ij}^p y_j, & \forall p\in l_1\\
                    & 0 \geq \sum_{i=1}^{n}v_i^r - \sum_{i=1}^{n} \sum_{j=1}^{n} u_{ij}^r y_j, & \forall r\in l_2\\
                    & y_{j} \in\{0,1\}, & \forall j\in\{1,\ldots,m\}
\end{align*}
$

**Exercice:** Implement the method `create_master_problem` that creates the initial master problem (without feasibility or optimality constraints) for the warehouse allocation problem.

<div class="alert alert-info alert-block">

You can use `print(m.export_as_lp_string())` to display a textual representation of a `docplex` model `m`.
    
</div>

In [13]:
from docplex.mp.model import Model
from docplex.mp.linear import Var
from docplex.mp.constr import AbstractConstraint
from typing import List, Sequence, Tuple


def create_master_problem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]]
) -> Tuple[Model, Var, Sequence[Var]]:
    """
    Creates the initial Benders master problem for the Warehouse Allocation problem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.

    Returns:
        A 3-tuple containing the docplex problem, the z variable and the y variables.
    """

    wa = Model("Warehouse Allocation - Benders master problem")
    z = wa.integer_var()
    y = wa.binary_var_list(M)
    wa.minimize(wa.dot(y, f)+z)
    return wa, z, y


# Check your method:
wa, z, y = create_master_problem(N, M, f, c)
print(wa.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: x1 + 20 x2 + 20 x3 + 20 x4
Subject To

Bounds
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x2 x3 x4

Generals
 x1
End



**Exercice:** Implement the method `add_optimality_constraints` that add optimality constraints to the Benders master problem. 

In [14]:
def add_optimality_constraint(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        wa: The Benders master problem (docplex.mp.model.Model).
        z: The z variable of the master problem.
        y: The y variables of the master problem.
        v: The optimal values for the v variables of the Benders dual subproblem.
        u: The optimal values for the u variables of the Benders dual subproblem.

    Return: The optimality constraint added.
    """
    c = wa.add_constraint(z >= wa.sum(v)-wa.sum([wa.dot(y, u[i]) for i in range(M)]))
    return c

**Exercice:** Implement the method `add_feasibility_constraints` that add feasibility constraints to the Benders master problem. 

In [15]:
def add_feasibility_constraints(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
      - N: Number of clients.
      - M: Number of warehouses.
      - wa: The Benders master problem (docplex.mp.model.Model).
      - z: The z variable of the master problem.
      - y: The y variables of the master problem.
      - v: The extreme rays for the v variables of the Benders dual subproblem.
      - u: The extreme rays for the u variables of the Benders dual subproblem.

    Returns:
        The feasibility constraint added.
    """
    c =  wa.add_constraint(0 >= wa.sum(v)-wa.sum([wa.dot(y, u[i]) for i in range(M)]))
    return c

**Exercice:** Implement the method `create_dual_subproblem` that, given a solution `y` of the master problem, create the corresponding Benders dual subproblem.

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

In [16]:
from docplex.mp.model import Model


def create_dual_subproblem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]], y: Sequence[int]
) -> Tuple[Model, Sequence[Var], Sequence[Sequence[Var]]]:
    """
    Creates a Benders dual subproblem for the Warehouse Allocation problem corresponding
    to the given master solution.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.
        y: Values of the y variables from the Benders master problem.

    Returns:
        A 3-tuple containing the docplex problem, the v variable and the u variables.
    """

    dsp = Model("Warehouse Allocation - Benders dual subproblem")

    # We disable pre-solve to be able to retrieve a meaningful status in the main
    # algorithm:
    dsp.parameters.preprocessing.presolve.set(0)

    #Variables
    v = dsp.continuous_var_list(N)
    u = [dsp.continuous_var_list(M, 0, name=f"X_{i}") for i in range(N)]

    #Constraints
    for i in range(N):
        for j in range(M):
            dsp.add_constraint(v[i] - u[i][j] <= c[i][j])

    #Objective
    dsp.maximize(dsp.sum(v)-dsp.sum([dsp.dot(u[i], y) for i in range(N)]))

    return dsp, v, u


# Check your method (assuming y = [1 1 1 ... 1]):
dsp, v, u = create_dual_subproblem(N, M, f, c, [1] * M)
print(dsp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders dual subproblem

Maximize
 obj: x1 + x2 + x3 - X_0_0 - X_0_1 - X_0_2 - X_1_0 - X_1_1 - X_1_2 - X_2_0
      - X_2_1 - X_2_2
Subject To
 c1: x1 - X_0_0 <= 15
 c2: x1 - X_0_1 <= 1
 c3: x1 - X_0_2 <= 2
 c4: x2 - X_1_0 <= 1
 c5: x2 - X_1_1 <= 16
 c6: x2 - X_1_2 <= 3
 c7: x3 - X_2_0 <= 4
 c8: x3 - X_2_1 <= 1
 c9: x3 - X_2_2 <= 17

Bounds
End



**Exercice:** Using the methods you implemented, write the Benders decomposition algorithm for the warehouse allocation problem.

<div class="alert alert-block alert-info">

The `get_extreme_rays` function can be used to retrieve the extreme rays associated with an unbounded solution of the dual subproblem.
    
</div>

<div class="alert alert-block alert-info">
    
You can use `model.get_solve_status()` to obtain the status of the resolution and compare it to members of `JobSolveStatus`:
    
```python
if model.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION:
    pass
```
    
</div>

In [18]:
from docplex.mp.model import Model
from docplex.util.status import JobSolveStatus


def get_extreme_rays(
    N: int, M: int, model: Model, v: Sequence[Var], u: Sequence[Sequence[Var]]
) -> Tuple[Sequence[float], Sequence[Sequence[float]]]:
    """
    Retrieves the extreme rays associated to the dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        model: The Benders dual subproblem model (docplex.mp.model.Model).
        v: 1D array containing the v variables of the subproblem.
        u: Either a 2D array of a tuple-index dictionary containing the u variables
            of the subproblem.

    Returns:
        A 2-tuple containing the list of extreme rays correspondig to v,
        and the 2D-list of extreme rays corresponding to u.
    """
    ray = model.get_engine().get_cplex().solution.advanced.get_ray()

    if isinstance(u, dict):

        def get_uij(i, j):
            return u[i, j]

    else:

        def get_uij(i, j):
            return u[i][j]

    return (
        [ray[v[i].index] for i in range(N)],
        [[ray[get_uij(i, j).index] for j in range(M)] for i in range(N)],
    )


# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

# We stop iterating if the new solution is less than epsilon
# better than the previous one:
epsilon = 1e-6

wa, z, y = create_master_problem(N, M, f, c)

n = 0
while True:

    # Print iteration:
    n = n + 1
    print("Iteration {}".format(n))

    dsp, v, u = create_dual_subproblem(N, M, f, c, solution.get_values(y))
    solution_dsp = dsp.solve()

print("Done.")

Iteration 1


DOcplexException: Cannot mix objects from different models in new_binary_constraint. obj1=X_0_1, obj2=x2

### 3.4. Generating instances for the Warehouse Allocation problem

**Exercice:** Using the TSP instances contained in `tsp.data` or the `generate_distances` method, create instances for the warehouse allocation problem with randomized opening costs.

In [19]:
#Variables
N = 10
M = 10
f = (int)(np.random.rand(N) * 100)
c = np.identity(N)

#Solution
solve_benders(N,M,f,c)

TypeError: only size-1 arrays can be converted to Python scalars

<div class="alert alert-block alert-danger"></div>